In [28]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [64]:
R = 4
N = 9

In [65]:
# Thetas0
Thetas0 = np.arange(R*(N-1)).reshape(R, N-1)
Thetas0[:, 0:2] = 0
Thetas0[:, 3:5] = 0
Thetas0[:, 6:] = 0
Thetas0

array([[ 0,  0,  2,  0,  0,  5,  0,  0],
       [ 0,  0, 10,  0,  0, 13,  0,  0],
       [ 0,  0, 18,  0,  0, 21,  0,  0],
       [ 0,  0, 26,  0,  0, 29,  0,  0]])

In [66]:
# Gams
Gams0Init = np.array([10, 2, 3, 9]).reshape(-1, 1)
tVec = np.concatenate([Gams0Init, Thetas0], axis=1)
Gams0 = np.cumsum(tVec, axis=1)
Gams0 

array([[10, 10, 10, 12, 12, 12, 17, 17, 17],
       [ 2,  2,  2, 12, 12, 12, 25, 25, 25],
       [ 3,  3,  3, 21, 21, 21, 42, 42, 42],
       [ 9,  9,  9, 35, 35, 35, 64, 64, 64]])

In [67]:
Gams0Vec = Gams0.T.reshape(-1)
Theta0Vec = Thetas0.T.reshape(-1)

### The data

In [68]:
Xs = np.random.rand(10, R*N)
Ys = Xs * Gams0Vec + np.random.randn(10, R*N)

### Opt

In [289]:
GamsVec = cp.Variable(R*N, value=np.random.randn(R*N))
ThetasVec = cp.Variable(R*(N-1), value=np.random.randn(R*(N-1)))
Gams = cp.reshape(GamsVec, (R, N))
Thetas = cp.reshape(ThetasVec, (R, N-1))
resid = GamsVec[R:] - GamsVec[:-R] - ThetasVec
lam = cp.Parameter(value=5000, nonneg=True)
g = lam/2 * cp.norm1(cp.norm(Thetas, axis=0))

In [290]:
rhos = cp.Parameter(R*(N-1), value=np.ones(R*(N-1)))
beta = 1

In [291]:
Num = Ys.shape[0]
f = 0
for i in range(Num):
    Y = Ys[i, :]
    X = Xs[i, :]
    f += cp.sum_squares(Y-cp.multiply(X, GamsVec))

In [292]:
aug_lagr1 = f + rhos.T@resid + (beta/2) * cp.sum_squares(resid)
aug_lagr2 = g + rhos.T@resid + (beta/2) * cp.sum_squares(resid)

In [293]:
for k in range(100):
    cp.Problem(cp.Minimize(aug_lagr1), [Thetas == Thetas.value]).solve("SCS")
    cp.Problem(cp.Minimize(aug_lagr2), [Gams == Gams.value]).solve("SCS")
    rhos.value += beta*resid.value

In [294]:
(rhos.T@resid).value

2279.2551061127756

In [295]:
Thetas0 - Thetas.value

array([[-3.37456953e-06, -3.76602141e-06,  2.00000607e+00,
        -3.93128153e-07,  2.34309968e-06,  5.00000294e+00,
         2.02389465e-06,  1.20324485e-07],
       [-1.25862716e-05, -1.10789036e-05,  1.00000138e+01,
         2.00759890e-06,  5.15689157e-06,  1.30000018e+01,
         1.13174841e-05,  3.54133105e-06],
       [-1.32312235e-05, -1.71939707e-05,  1.80000323e+01,
         5.50000993e-07,  1.32371308e-05,  2.10000173e+01,
         1.93742109e-05,  3.88488979e-06],
       [-3.13600229e-05, -1.62356964e-05,  2.60000296e+01,
         7.11259697e-06,  1.21976873e-05,  2.90000090e+01,
         2.37753231e-05,  9.23461016e-06]])

In [296]:
Thetas.value

array([[ 3.37456953e-06,  3.76602141e-06, -6.06999485e-06,
         3.93128153e-07, -2.34309968e-06, -2.93506294e-06,
        -2.02389465e-06, -1.20324485e-07],
       [ 1.25862716e-05,  1.10789036e-05, -1.37923717e-05,
        -2.00759890e-06, -5.15689157e-06, -1.84452889e-06,
        -1.13174841e-05, -3.54133105e-06],
       [ 1.32312235e-05,  1.71939707e-05, -3.23401573e-05,
        -5.50000993e-07, -1.32371308e-05, -1.73457108e-05,
        -1.93742109e-05, -3.88488979e-06],
       [ 3.13600229e-05,  1.62356964e-05, -2.95662183e-05,
        -7.11259697e-06, -1.21976873e-05, -9.00925690e-06,
        -2.37753231e-05, -9.23461016e-06]])

In [267]:
Thetas0

array([[ 0,  0,  2,  0,  0,  5,  0,  0],
       [ 0,  0, 10,  0,  0, 13,  0,  0],
       [ 0,  0, 18,  0,  0, 21,  0,  0],
       [ 0,  0, 26,  0,  0, 29,  0,  0]])

In [297]:
np.diff(Gams.value, axis=1) - Thetas.value

array([[ 3.45268622e-02,  2.73395632e-01, -9.25824449e-02,
         3.27592827e-01,  6.61045925e-01, -5.85025474e-01,
         2.22327408e-01, -8.69308582e-04],
       [ 2.00050869e-01,  5.87964148e-01, -3.62491621e-01,
         1.43806740e+00,  1.21466380e+00, -9.72719841e-01,
         1.25862680e+00,  3.33575841e-01],
       [ 2.31962903e-01,  9.49406711e-01, -1.58971398e+00,
         1.60130230e+00,  1.71395354e+00, -2.14363324e+00,
         1.21902491e+00,  6.15216005e-01],
       [ 6.82673051e-01,  1.43383717e+00, -1.05450121e+00,
         2.42209104e+00,  1.96806795e+00, -2.19287921e+00,
         1.31460394e+00,  3.55287562e-01]])